# Import Modules

## Standard

In [ ]:
import sys
import os
import os.path as path
import xarray as xr
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## User Defined

In [ ]:
sys.path.insert(0, os.path.dirname(os.getcwd()))
from data_processing_slice import *
from plotting_slice import *
from helper_GAD import *

# Paths

In [ ]:
WRF_result_base_loc ='/p/lustre2/jha3/fromWill/TurbTest/NREL_5MW'
GAD_param_output_loc = os.path.join(WRF_result_base_loc, 'GAD_Param_Study_Output')
GAD_param_power_curve_loc = os.path.join(GAD_param_output_loc, 'PowerCurve')
GAD_param_slice_loc = os.path.join(GAD_param_output_loc, 'Slice')
os.system('mkdir -p %s'%GAD_param_power_curve_loc)
os.system('mkdir -p %s'%GAD_param_slice_loc)

# Flags

In [ ]:
savefig = True
induction_effect = True
grid_effect = False
epsilon_effect = False

# Common Stuff

In [ ]:
hub_height = 90
vhub = [3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5, 15]
#vhub = [3, 4.5, 6, 7.5, 9, 10.5, 13.5, 15, 18]
ind1_for_tab = 3 # To be used in the paper
ind2_for_tab = 4 # To be used in the paper
ind3_for_tab = 5 # To be used in the paper
outfile = 'wrfout_d01_0001-01-01_00:00:00'
tsoutfile = 'tsout_d01_0001-01-01_00:00:00'

# Effect of Induction Type

In [ ]:
if induction_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_induction.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_induction.csv')
	plt_title   = '$Grid: \Delta x = \Delta y = 4 m, \Delta z = 4 m; Gaussian: \epsilon/\Delta_{grid} = 1.00 $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': 
                    {'legend':'Glauert, a$_{n}$ = 0.07, a$_{t}$ = iterated', 
                     'line_style': 'b',
                     'dz': 4.0},
		        'Glauert_Tip_dx_04_dz_04_an_iter_at_iter_eps_1.00': 
                    {'legend':'Glauert, a$_{n}$ = iterated, a$_{t}$ = iterated', 
                     'line_style': 'g',
                     'dz': 4.0},
			'Shen_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': 
                    {'legend':'Shen, a$_{n}$ = 0.07, a$_{t}$ = iterated', 
                     'line_style': 'r--',
                     'dz': 4.0},
			'Shen_Tip_dx_04_dz_04_an_iter_at_iter_eps_1.00': 
                    {'legend':'Shen, a$_{n}$ = iterated, a$_{t}$ = iterated', 
                     'line_style': 'm--',
                     'dz': 4.0},
			'Tip_dx_04_dz_04_an_0.0_at_0.0_eps_1.00': 
                    {'legend':'a$_{n}$ = 0.0, a$_{t}$ = 0.0', 
                     'line_style': 'c',
                     'dz': 4.0}}

# Effect of Grid Resolution

In [ ]:
if grid_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_grid_effect.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_grid_effect.csv')
	plt_title   = '$ Induction: Glauert, a_{n} = 0.07, a_{t} = iterated; Gaussian: \epsilon/\Delta_{grid} = 1.00 $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': 
                    {'legend':'$\Delta x = \Delta y = 4 m, \Delta z = 4 m $', 
                     'line_style': 'b',
                     'dz': 4.0},
			'Glauert_Tip_dx_08_dz_08_an_fixed_at_iter_eps_1.00': 
                    {'legend':'$\Delta x = \Delta y = 8 m, \Delta z = 8 m $', 
                     'line_style': 'g',
                     'dz': 8.0},
			'Glauert_Tip_dx_16_dz_04_an_fixed_at_iter_eps_1.00': 
                    {'legend':'$\Delta x = \Delta y = 16 m, \Delta z = 4 m $', 
                     'line_style': 'r--',
                     'dz': 4.0},
			'Glauert_Tip_dx_16_dz_16_an_fixed_at_iter_eps_1.00': 
                    {'legend':'$\Delta x = \Delta y = 16 m, \Delta z = 16 m $', 
                     'line_style': 'm--',
                     'dz': 16.0}}

# Effect of Gaussian Spreading

In [ ]:
if epsilon_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_gaussian_effect.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_gaussian_effect.csv')
	plt_title   = '$Grid: \Delta x = \Delta y = 4 m, \Delta z = 4 ; Induction: Glauert, a_{n} = 0.07, a_{t} = iterated $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_0.707': 
                    {'legend':'$ \epsilon/\Delta_{grid} = 1/\sqrt{2} $', 
                     'line_style': 'b',
                     'dz': 4.0},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': 
                    {'legend':'$ \epsilon/\Delta_{grid} = 1.00$', 
                     'line_style': 'g',
                     'dz': 4.0},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.4142': 
                    {'legend':'$ \epsilon/\Delta_{grid} = \sqrt{2} $', 
                     'line_style': 'r--',
                     'dz': 4.0},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_2.00': 
                    {'legend':'$ \epsilon/\Delta_{grid} = 2.00 $', 
                     'line_style': 'm--',
                     'dz': 4.0}}

# Cases of Interest

In [ ]:
case_keys = list(case_dir_map.keys())
case_keys.sort()
print ('Cases: \n {}'.format(case_keys))
#case_keys

# Compute Indices and Weights for Slices at Hub Height

In [ ]:
case_dir_map = compute_indices_weights_for_slices (case_dir_map, hub_height)
#case_dir_map

# NREL Data

In [ ]:
NREL_Data, ws_NREL, power_NREL, power_interp = prepare_NREL_data (WRF_result_base_loc, vhub)

# Initialize Tabulated Data

In [ ]:
case_tab, power1_tab, power2_tab, power3_tab, \
error1_tab, error2_tab, error3_tab = \
                initialize_tabulated_data (power_interp, \
                                           ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Read the Power Data for Cases of Interest

In [ ]:
print ('Cases under consideration : \n')
for case in case_keys:
    case_dir_map, case_tab, \
    power1_tab, power2_tab, power3_tab, \
    error1_tab, error2_tab, error3_tab = \
    read_power_for_a_case (WRF_result_base_loc, case, vhub, power_interp, outfile, \
                           case_dir_map, case_tab, \
                           power1_tab, power2_tab, power3_tab, \
                           error1_tab, error2_tab, error3_tab, \
                           ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Prepare Tabulated Data for a Few Wind Speeds

In [ ]:
tabulate_data_for_cases (tab_data_file, vhub, case_tab, \
                         power1_tab, power2_tab, power3_tab, \
                         error1_tab, error2_tab, error3_tab, \
                         ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Create Power Curve Plots

## Power Curve

In [ ]:
plot_power_curve_for_cases (NREL_Data, vhub, case_keys, case_dir_map, plt_title, \
                            figFileName, savefig)

## Error in Power Curve

In [ ]:
plot_power_curve_error_for_cases (vhub, case_keys, case_dir_map, plt_title)

# Read and Plot Slice Data

In [ ]:
case_dir_map

In [ ]:
ws_list = [9]
qoi_label_map = {'UTS':'U'}
#qoi_label_map = {'UTS':'U','VTS':'V'}
qoi_unit_map = {'UTS':'m/s', 'VTS':'m/s'}
time_ind = 30
qoi_cont_range = [2, 11]
cont_levels_count = 21
xlim = [0, 1000]
ylim = [0, 1000]

In [ ]:
for case in case_keys:
    case_legend = case_dir_map[case]['legend']
    DZ = case_dir_map[case]['dz']
    z_ind1 = case_dir_map[case]['z_indices'][0]
    z_ind2 = case_dir_map[case]['z_indices'][1]
    w1 = case_dir_map[case]['weights'][0]
    w2 = case_dir_map[case]['weights'][1]
    print('z_indices: [{}, {}], weights: [{}, {}]'.format(z_ind1, z_ind2, w1, w2))
    
    for ws in ws_list:

        slice_data_wgt_UMAG, DX, DY = \
        get_hub_height_slice_data_case_ws (WRF_result_base_loc, case, ws, outfile, tsoutfile, \
                                      time_ind, z_ind1, z_ind2, w1, w2)
        
        plot_loc = os.path.join(GAD_param_slice_loc, case, 'power_curve_{}'.format(ws))
        os.system('mkdir -p %s'%plot_loc)
        image_name = 'slice_{}.png'.format('UMAG')

        plot_contour_slice(slice_data_wgt_UMAG, DX, DY, plot_loc, image_name, \
                            'UMAG', 'UMAG', 'm/s', case_legend, \
                            cont_levels_count, qoi_cont_range)
     

# Scratch Work